<h2> 異分野共著　process I : Make ID list </h2>

In [ ]:
#! Python3
# -*- coding: utf-8 -*-

import csv, os, glob
import pandas  as pd
import numpy as np


<h3>0.事前準備</h3>

In [ ]:
#複数ファイルがある場合: fromSciValというフォルダを作り、SciValからDLしたファイルをそのまま入れておく
#DLした項目によって、下記を変更
col_names =  ['Title', 'Authors', 'Number of Authors', 'Scopus Author Ids', 'Year', 'Scopus Source title', 'SNIP 2017', 'Citations',
       'Field-Weighted Outputs in Top Citation Percentiles, per percentile', 'DOI', 'Publication-type', 'EID',
       'All Science Journal Classification (ASJC)']

#fromSciVal内にあるファイルの名を変更、一つのリストにまとめる
publicationsAll = pd.DataFrame(index=[], columns=col_names)
# カレントディレクトリの全ファイルをループする
for csv_file in os.listdir('fromSciVal/.'):
    if not csv_file.endswith('.csv'):
        continue # CSVファイルでなければスキップ

    print('Change name: ' + csv_file + '...')
    df = pd.read_csv('fromSciVal/' + csv_file, header = None, dtype ='object',names = col_names)
    dfname = str(df.iat[0,1] + "_" +df.iat[1,1])
    os.rename('fromSciVal/' + csv_file, 'fromSciVal/' + dfname + '_190207.csv')
    df2 = df.dropna(axis = 0, subset=['All Science Journal Classification (ASJC)'])
    df3= df2[df2['Title'] != 'Title']
#    df3['Scopus Author Ids'] =  df3['Scopus Author Ids'].str.replace("|",",")
    print(df3['Scopus Author Ids'])
    
    
    publicationsAll = pd.concat([publicationsAll, df3], ignore_index=True)

# 出力するファイル名を指定する。
publicationsAll.to_csv('Publications_KU2020.csv', index = False)


<h3> 1. publicationsAllより、重複削除、単著論文を除外、HAPは著者情報が500までしかDLされないが、とりあえずそのまま（APIで確認し直す）</h3>

In [ ]:
publicationsAll['Number of Authors'] = publicationsAll['Number of Authors'].astype(int)
uniquepubdf = publicationsAll.drop_duplicates()
targetdf = uniquepubdf[uniquepubdf['Number of Authors'] > 1]
print('all publications:' + str(len(uniquepubdf)))
print('Target publications (単著以外):' + str(len(targetdf)))

In [ ]:
targetdf.columns


<h3>2. Author ID を抽出(重複あり)、数を確認</h3>

In [ ]:
aIds = []
nofaIds = 0
for i, row in targetdf.iterrows():
    author = [row['Scopus Author Ids']]
    author = author[0].split('| ')
    aIds.extend(author)
    if row['Number of Authors'] > 500:
        nofa = 500
    else:
        nofa = row['Number of Authors']
    nofaIds = nofaIds + nofa

    #HAP以外でIDの数とAuthor数が一致しない場合、そのscopusIDを出力（一応確認のため）
    if len(author) != nofa:
        print(row['EID'])
    else:
        continue
        
aIdsdf = pd.DataFrame(aIds, columns=['Author ID'])

#Authorの数とIDの数が一致するか確認
print('Author Ids count: ' + str(len(aIdsdf)))
print('Number of Authors: ' + str(nofaIds))


<h3>3. Unique Author ID listを作る</h3>

In [ ]:
uniqueaIds = aIdsdf.drop_duplicates()
uniqueaIds =uniqueaIds.astype(int)
print('Unique Author IDs: ' + str(len(uniqueaIds)))

In [ ]:
uniqueaIds.to_csv('UniqueAuthorIdsList.csv', index = False)